In [ ]:
# | default_exp mkdocs


In [ ]:
# | export

from typing import *

import os
import re
import collections
from pathlib import Path
import textwrap
import shutil
import types
import pkgutil
import importlib
import subprocess  # nosec: B404
import shlex
import sys
import multiprocessing
import datetime
from tempfile import TemporaryDirectory
import yaml

import typer
from typer.testing import CliRunner

from configupdater import ConfigUpdater, Section
from configupdater.option import Option

from configparser import ConfigParser
from fastcore.script import call_parse

import nbdev
from nbdev.serve import proc_nbs
from nbdev.process import NBProcessor
from nbdev.frontmatter import FrontmatterProc
from nbdev.quarto import prepare as nbdev_prepare
from nbdev.quarto import refresh_quarto_yml, nbdev_readme
from nbdev.doclinks import nbdev_export
from nbdev.frontmatter import _fm2dict
from fastcore.shutil import move

from nbdev_mkdocs._package_data import get_root_data_path
from nbdev_mkdocs._helpers.cli_doc import generate_cli_doc
from nbdev_mkdocs._helpers.utils import set_cwd, get_value_from_config
from nbdev_mkdocs.social_image_generator import _update_social_image_in_mkdocs_yml


In [ ]:
import pytest
import numpy as np
import unittest.mock
from ruamel.yaml import YAML

from nbdev.config import nbdev_create_config


## Create new

### Add requirements to settings

In [ ]:
# | export


def _add_requirements_to_settings(root_path: str):
    """Adds requirments needed for mkdocs to settings.ini

    Args:
        root_path: path to where the settings.ini file is located

    """
    _requirements_path = get_root_data_path() / "requirements.txt"
    with open(_requirements_path, "r") as f:
        _new_req_to_add = f.read()
        lines = _new_req_to_add.split("\n")
        lines = [s.strip() for s in lines]
        lines = [s for s in lines if s != ""]
        _new_req_to_add = " \\\n".join(lines)

    setting_path = Path(root_path) / "settings.ini"
    if not setting_path.exists():
        typer.secho(
            f"Path '{setting_path.resolve()}' does not exists! Please use --root_path option to set path to setting.ini file.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:

        updater = ConfigUpdater()
        updater.read(setting_path)
    except Exception as e:
        typer.secho(
            f"Error while reading '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=2)

    try:
        _user = updater["DEFAULT"]["user"].value
        _repo = updater["DEFAULT"]["repo"].value
        option_name = (
            "requirements"
            if (f"{_user}/{_repo}") == "airtai/nbdev-mkdocs"
            else "dev_requirements"
        )
        if option_name not in updater["DEFAULT"]:
            updater["DEFAULT"].last_block.add_after.space(2).comment(f"### {option_name.title()} ###").option(option_name, "")  # type: ignore

        old_req: str = updater["DEFAULT"][option_name].value  # type: ignore

        def remove_leading_spaces(s: str) -> str:
            return "\n".join([x.lstrip() for x in s.split("\n")])

        old_req = remove_leading_spaces(old_req)
        new_req = remove_leading_spaces(_new_req_to_add)
        if new_req in old_req:
            typer.secho(f"Requirements already added to '{setting_path.resolve()}'.")
            return

        req = old_req + " \\\n" + new_req
        req = textwrap.indent(req, " " * 4)

        req_option = Option(key=option_name, value=req)
        updater["DEFAULT"][option_name] = req_option
    except Exception as e:
        typer.secho(
            f"Error while updating requiremets in '{setting_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

    updater.update_file()

    typer.secho(f"Requirements added to '{setting_path.resolve()}'.")

    return


In [ ]:
with TemporaryDirectory() as d:
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["requirements"] = Option(
        key="requirements", value="\\\n  nbdev>=2.3.7 \\\n  typer[all]==0.6.1"
    )
    updater.update_file()

    assert "mkdocs" not in updater["DEFAULT"]["requirements"].value

    # testing adding requirements
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    # do nothin if the requirements are already added
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall("mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["requirements"].value)
    assert len(founded) == 5, founded

    print(updater)


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp72_ay_r5/settings.ini'.
Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp72_ay_r5/settings.ini'.
[DEFAULT]
# All sections below are required unless otherwise specified.
# See https://github.com/fastai/nbdev/blob/master/settings.ini for examples.

### Python library ###
repo = nbdev-mkdocs
lib_name = %(repo)s
version = 0.0.2rc1
min_python = 3.7
license = apache2

### nbdev ###
doc_path = _docs
lib_path = nbdev_mkdocs
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
black_formatting = True

### Docs ###
branch = main
custom_sidebar = False
doc_host = https://%(user)s.github.io
doc_baseurl = /%(repo)s
git_url = https://github.com/%(user)s/%(repo)s
title = %(lib_name)s

### PyPI ###
audience = Developers
author = airt
author_email = info@airt.ai
copyright = 2022 onwards, %(author)s
description = Extension of nbdev for generating documenta

In [ ]:
with TemporaryDirectory() as d:
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["repo"] = "some-random-repo-name"
    updater["DEFAULT"]["requirements"] = Option(
        key="requirements", value="\\\n  nbdev>=2.3.7 \\\n  typer[all]==0.6.1"
    )
    updater.update_file()

    assert "mkdocs" not in updater["DEFAULT"]["requirements"].value

    # testing adding requirements
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall(
        "mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["dev_requirements"].value
    )
    assert len(founded) == 5, founded

    # do nothin if the requirements are already added
    _add_requirements_to_settings(d)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    founded = re.findall(
        "mkdocs[\w_\-\[\]]*", updater["DEFAULT"]["dev_requirements"].value
    )
    assert len(founded) == 5, founded

    print(updater)


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbsn7q4rn/settings.ini'.
Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbsn7q4rn/settings.ini'.
[DEFAULT]
# All sections below are required unless otherwise specified.
# See https://github.com/fastai/nbdev/blob/master/settings.ini for examples.

### Python library ###
repo = some-random-repo-name
lib_name = %(repo)s
version = 0.0.2rc1
min_python = 3.7
license = apache2

### nbdev ###
doc_path = _docs
lib_path = nbdev_mkdocs
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
black_formatting = True

### Docs ###
branch = main
custom_sidebar = False
doc_host = https://%(user)s.github.io
doc_baseurl = /%(repo)s
git_url = https://github.com/%(user)s/%(repo)s
title = %(lib_name)s

### PyPI ###
audience = Developers
author = airt
author_email = info@airt.ai
copyright = 2022 onwards, %(author)s
description = Extension of nbdev for generating 

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str):
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        typer.secho(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs created.
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/site_overrides
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/.ipynb_checkpoints
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/docs_overrides
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/site_overrides/main.html
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/site_overrides/partials
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/site_overrides/partials/copyright.html
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/docs_overrides/css
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/docs_overrides/images
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/docs_overrides/js
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvpjhh_b9/mkdocs/docs_overrides/css/extra.css


### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"


In [ ]:
assert _mkdocs_template_path.exists()


In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()


In [ ]:
print(_mkdocs_template)


# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences
    - pymdownx.tasklist
    - pymdownx.highlight:
        linenums: false
    - pymdownx.snippets:
        check_paths: true
    - pymdownx.t

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs


{'title': 'nbdev-mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev-mkdocs',
 'author': 'airt',
 'description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo': 'nbdev-mkdocs',
 'git_url': 'https://github.com/airtai/nbdev-mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str):
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

y


File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpgysj8w0g/mkdocs/mkdocs.yml' generated.


{'site_name': 'nbdev-mkdocs',
 'site_url': 'https://airtai.github.io/nbdev-mkdocs',
 'site_author': 'airt',
 'site_description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo_name': 'nbdev-mkdocs',
 'repo_url': 'https://github.com/airtai/nbdev-mkdocs',
 'edit_uri': '',
 'copyright': '2022 onwards, airt',
 'docs_dir': 'docs',
 'site_dir': 'site',
 'plugins': [{'literate-nav': {'nav_file': 'SUMMARY.md'}},
  'search',
  {'mkdocstrings': {'handlers': {'python': {'import': ['https://docs.python.org/3/objects.inv'],
      'options': {'heading_level': 2,
       'show_category_heading': True,
       'show_root_heading': True,
       'show_root_toc_entry': True,
       'show_signature_annotations': True,
       'show_if_no_docstring': True}}}}}],
 'markdown_extensions': [{'pymdownx.arithmatex': {'generic': True}},
  'pymdownx.inlinehilite',
  'pymdownx.details',
  'pymdownx.emoji',
  'pymdownx.magiclink',
  'pymdownx.superfences',
  'pymdo

### Create summary_template.txt

In [ ]:
# | export

_summary_template = """{sidebar}
{api}
{cli}
{changelog}
"""


def _create_summary_template(root_path: str):
    try:
        # create mkdocs folder if necessary
        summary_template_path = Path(root_path) / "mkdocs" / "summary_template.txt"
        summary_template_path.parent.mkdir(exist_ok=True)
        # summary_template_path.yml already exists, just return
        if summary_template_path.exists():
            typer.secho(
                f"Path '{summary_template_path.resolve()}' exists, skipping generation of it."
            )
            return

        # generated a new summary_template_path.yml file
        with open(summary_template_path, "w") as f:
            f.write(_summary_template)
            typer.secho(f"File '{summary_template_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{summary_template_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)
    _create_summary_template(d)

    sidebar = """- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)"""

    api = """- API
    - [numpy.array](api/numpy/array.md)"""

    cli = """- CLI
    - [my-cli](cli/my_cli.md)"""

    changelog = "- [Releases](CHANGELOG.md)"

    with open(Path(d) / "mkdocs/summary_template.txt") as f:
        summary_template = f.read()
        summary = summary_template.format(
            sidebar=sidebar, api=api, cli=cli, changelog=changelog
        )
#         y = yaml.safe_load(summary)

print(summary)


File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mnmnd5z/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4mnmnd5z/mkdocs/summary_template.txt' generated.
- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)



In [ ]:
# | export


def _replace_ghp_deploy_action(root_path: str):
    """Replace the default gh-pages deploy action file with the custom action template file

    Args:
        root_path: Project's root path
    """

    src_path = get_root_data_path() / "ghp_deploy_action_template.yml"
    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)

    workflows_path = Path(root_path) / ".github" / "workflows"
    workflows_path.mkdir(exist_ok=True, parents=True)

    dst_path = Path(workflows_path) / "deploy.yaml"
    shutil.copyfile(src_path, dst_path)


In [ ]:
with TemporaryDirectory() as d:
    assert not (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()
    _replace_ghp_deploy_action(d)
    assert (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()


In [ ]:
def run_nbdev_new(d):

    path = Path(".") if Path("settings.ini").exists() else Path("..")
    Path(Path(d) / "nbs").mkdir(exist_ok=True)

    # Create a sample .gitignore file
    gitignore_path = Path(d) / ".gitignore"
    sample_git_ignore = (
        "_docs/"
        "_proc/\n"
        "*.bak\n"
        "# Byte-compiled / optimized / DLL files\n"
        "__pycache__/"
    )

    with gitignore_path.open("w", encoding="utf-8") as f:
        f.write(sample_git_ignore)

    assert gitignore_path.exists()

    # copy nbdev-mkdocs/nbs/index.ipynb folder to {d}/nbs/index.ipynb
    for fname in ["index.ipynb"]:
        shutil.copyfile(path / "nbs" / fname, Path(d) / "nbs" / fname)

    # copy config files from nbdev-mkdocs/ to {d}/
    for fname in ["setup.py"]:
        shutil.copyfile(path / fname, Path(d) / fname)

    # Generate settings.ini using nbdev in {d}
    with set_cwd(d):
        nbdev_create_config(
            repo="repo",
            branch="branch",
            user="user",
            author="author",
            author_email="author@mail.com",
            description="description",
        )
        refresh_quarto_yml()
        nbdev_export.__wrapped__()
        nbdev_readme.__wrapped__(chk_time=True)


In [ ]:
# | export


def _update_gitignore_file(root_path: str):
    """Update the .gitignore file to include the autogenerated mkdocs directories

    Args:
        root_path: Project's root path
    """

    _mkdocs_gitignore_path = get_root_data_path() / "gitignore.txt"
    with open(_mkdocs_gitignore_path, "r") as f:
        _new_paths_to_ignore = f.read()
        _new_paths_to_ignore = "\n\n" + _new_paths_to_ignore

    gitignore_path = Path(root_path) / ".gitignore"
    if not gitignore_path.exists():
        typer.secho(
            f"Unexpected error: path {gitignore_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    with open(gitignore_path, "a") as f:
        f.write(_new_paths_to_ignore)


In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    gitignore_path = Path(d) / ".gitignore"

    with open(gitignore_path) as f:
        contents = f.read()
        assert "mkdocs/docs/" not in contents
        assert "mkdocs/site/" not in contents

    _update_gitignore_file(d)

    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/


Output created: _docs/README.md



In [ ]:
# | export


def _generate_default_social_image_link(root_path: str):
    """Generating default social sharing image link and add it to the mkdocs yaml file

    Args:
        root_path: Project's root path
    """

    with set_cwd(root_path):
        repo = get_value_from_config(root_path, "repo")
        user = get_value_from_config(root_path, "user")

        timestamp = datetime.datetime.now().timestamp()
        img_url = f"https://opengraph.githubassets.com/{timestamp}/{user}/{repo}"

        _update_social_image_in_mkdocs_yml(root_path, img_url)


In [ ]:
with TemporaryDirectory() as d:
    
    run_nbdev_new(d)
    
    _add_requirements_to_settings(d)
    _create_mkdocs_dir(d)
    _create_mkdocs_yaml(d)
    _create_summary_template(d)
    _replace_ghp_deploy_action(d)
    _update_gitignore_file(d)
    
    _generate_default_social_image_link(d)
    
    
    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

    assert y['extra']['social_image'] != ""

    !cat {d}/mkdocs/mkdocs.yml


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5l6nqrns/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5l6nqrns/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5l6nqrns/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5l6nqrns/mkdocs/summary_template.txt' generated.
# Site
site_name: repo
site_url: https://user.github.io/repo
site_author: author
site_description: description

# Repository
repo_name: repo
repo_url: https://github.com/user/repo
edit_uri: ''

copyright: 2022 onwards, author

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
        - https://docs.python.org/3/objects.inv
        options:
          heading_level: 2
          show_category_heading: true
          show_root_heading: true
          show_root_toc_entry: true
 

Output created: _docs/README.md



### Bringing it all together

In [ ]:
# | export


def new(root_path: str):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Args:
        root_path: path under which mkdocs directory will be created
    """
    _add_requirements_to_settings(root_path)
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)
    _create_summary_template(root_path)
    _replace_ghp_deploy_action(root_path)
    _update_gitignore_file(root_path)
    _generate_default_social_image_link(root_path)


@call_parse
def new_cli(root_path: str = "."):
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.
    """
    new(root_path)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    mkdocs_path = Path(d) / "mkdocs"

    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "site_overrides" / "main.html").exists()
    assert (mkdocs_path / "site_overrides" / "partials" / "copyright.html").exists()
    assert (mkdocs_path / "summary_template.txt").exists()

    gitignore_path = Path(d) / ".gitignore"
    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents

    with open(Path(d) / "mkdocs/mkdocs.yml") as f:
        y = yaml.safe_load(f)

    assert y["extra"]["social_image"] != ""

#     !ls {d}
#     !ls {d}/nbs


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpejnc3sdv/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpejnc3sdv/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpejnc3sdv/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpejnc3sdv/mkdocs/summary_template.txt' generated.
_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/


Output created: _docs/README.md



## Build

### Build markdown files

In [ ]:
# | export


def is_library_notebook(fname: Path) -> bool:
    """Check if a notebook is exported as part of the library

    Args:
        fname: The path to the notebook to check.

    Returns:
        `True` if the notebook is exported as part of the library, `False` otherwise.
    """
    if fname.suffix == ".qmd":
        return False

    nb = NBProcessor(fname)
    for cell in nb.nb.cells:
        if cell["cell_type"] == "code":
            if "default_exp" in cell["directives_"]:
                return True
    return False


def _get_files_to_convert_to_markdown(root_path: str) -> List[Path]:
    """Gets a list of notebooks and qmd files that need to be converted to markdown.

    Args:
        cache: Project's root path

    Returns:
        A list of files that need to be converted to markdown
    """
    with TemporaryDirectory() as d:
        nbs_directory = get_value_from_config(root_path, "nbs_path")
        src_nbs_path = Path(root_path) / nbs_directory

        dst_nbs_path = Path(d) / f"{nbs_directory}"
        shutil.copytree(src_nbs_path, dst_nbs_path)

        exts = [".ipynb", ".qmd"]
        files = [
            f
            for f in dst_nbs_path.rglob("*")
            if f.suffix in exts
            and not str(f.name).startswith("_")
            and not any(p.startswith(".") for p in f.parts)
            and not is_library_notebook(f)
        ]
        files = [f.relative_to(dst_nbs_path) for f in files]

        return files


In [ ]:
def create_sample_qmd_file(d):
    qmd_str = """---
title: "Sample"
author: Sample
date: last-modified
---

## Introduction

"""
    qmd_index_path = Path(d) / "nbs" / "sample.qmd"
    with open(qmd_index_path, "w") as f:
        f.write(qmd_str)


with TemporaryDirectory() as d:

    run_nbdev_new(d)

    (Path(d) / "nbs" / "guides").mkdir(exist_ok=True)
    (Path(d) / "nbs" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "nbs" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        (Path(d) / "nbs" / f"{i}" / ".ipynb_checkpoints").mkdir(exist_ok=True)
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "nbs" / f"{i}" / f"qmd_{i}.qmd",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / ".ipynb_checkpoints" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / f"_{i}_index.ipynb",
        )

    new(d)

    with set_cwd(d):
        nbs = _get_files_to_convert_to_markdown(Path(d))

    nbs = [str(nb) for nb in nbs]
    print(nbs)

    assert f"index.ipynb" in nbs
    assert f"sample.qmd" in nbs

    assert f"guides/guides_index.ipynb" in nbs
    assert f"guides/qmd_guides.qmd" in nbs

    assert f"blogs/blogs_index.ipynb" in nbs
    assert f"blogs/qmd_blogs.qmd" in nbs

    assert f"Mkdocs.ipynb" not in nbs
    assert f"_quarto.yml" not in nbs

    assert f"guides/.ipynb_checkpoints/guides_index.ipynb" not in nbs
    assert f"blogs/.ipynb_checkpoints/blogs_index.ipynb" not in nbs

    assert f"guides/_guides_index.ipynb" not in nbs
    assert f"blogs/_blogs_index.ipynb" not in nbs


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp57xqn4ta/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp57xqn4ta/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp57xqn4ta/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp57xqn4ta/mkdocs/summary_template.txt' generated.
['sample.qmd', 'index.ipynb', 'blogs/blogs_index.ipynb', 'blogs/qmd_blogs.qmd', 'guides/guides_index.ipynb', 'guides/qmd_guides.qmd']


Output created: _docs/README.md



In [ ]:
# | export


def _sprun(cmd):
    try:
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        subprocess.check_output(
            cmd, shell=True  # nosec: B602:subprocess_popen_with_shell_equals_true
        )

    except subprocess.CalledProcessError as e:
        sys.exit(
            f"CMD Failed: e={e}\n e.returncode={e.returncode}\n e.output={e.output}\n e.stderr={e.stderr}\n cmd={cmd}"
        )


In [ ]:
# | export


def _generate_markdown_from_files(root_path: str):

    doc_path = Path(root_path) / "mkdocs" / "docs"
    doc_path.mkdir(exist_ok=True, parents=True)

    with set_cwd(root_path):
        files = _get_files_to_convert_to_markdown(root_path)
        cache = proc_nbs()

        for f in files:
            dir_prefix = str(Path(f).parent)
            dst_md = doc_path / f"{dir_prefix}" / f"{f.stem}.md"
            dst_md.parent.mkdir(parents=True, exist_ok=True)

            cmd = f'cd "{cache}" && quarto render "{cache / f}" -o "{f.stem}.md" -t gfm --no-execute'
            _sprun(cmd)

            src_md = cache / "_docs" / f"{f.stem}.md"
            shutil.move(src_md, dst_md)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    (Path(d) / "nbs" / "guides").mkdir(exist_ok=True)
    (Path(d) / "nbs" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "nbs" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "nbs" / f"{i}" / f"qmd_{i}.qmd",
        )

    new(d)

    _generate_markdown_from_files(d)

    # check markdown files
    print("Checks:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    mds = [str(md) for md in mds]
    print("\n".join(mds))
    assert len(mds) > 0, len(mds)

    for i in [
        "index.md",
        "sample.md",
        "guides/guides_index.md",
        "guides/qmd_guides.md",
        "blogs/blogs_index.md",
        "blogs/qmd_blogs.md",
    ]:
        assert f"{d}/mkdocs/docs/{i}" in mds, f"{d}/mkdocs/docs/{i}"

    assert f"{d}/mkdocs/docs/Mkdocs.md" not in mds


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

pandoc -o sample.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
metadata
  title: Sample
  author: Sample
  date: last-modified
  
Output created: _docs/sample.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../blogs_index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: blogs_index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: ../_docs/blogs_index.md

pandoc -o ../qmd_blogs.md
  to: >-
    commonm

Checks:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/sample.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/index.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/blogs/qmd_blogs.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/blogs/blogs_index.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/guides/qmd_guides.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp16u57_ke/mkdocs/docs/guides/guides_index.md


Output created: ../_docs/qmd_guides.md



In [ ]:
# | export


def _replace_all(text: str, dir_prefix: str) -> str:
    """Replace the images relative path in the markdown text

    Args:
        text: String to replace
        dir_prefix: Sub directory prefix to append to the image's relative path

    Returns:
        The text with the updated images relative path
    """
    _replace = {}
    _pattern = re.compile(r"!\[[^\]]*\]\(([^https?:\/\/].*?)\s*(\"(?:.*[^\"])\")?\s*\)")
    _matches = [match.groups()[0] for match in _pattern.finditer(text)]

    if len(_matches) > 0:
        for m in _matches:
            _replace[m] = (
                os.path.normpath(Path("../images/nbs/").joinpath(f"{dir_prefix}/{m}"))
                if len(dir_prefix) > 0
                else f"images/nbs/{m}"
            )

        for k, v in _replace.items():
            text = text.replace(k, v)

    return text


In [ ]:
text = """![Git Repo_Clone_Page](../img/test.png)
![Git Repo_Clone_Page](images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

expected = """![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

dir_prefix = "guides"
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected

text = """![Git Repo_Clone_Page](img/test.png)
![Git Repo_Clone_Page](guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

expected = """![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
"""

dir_prefix = ""
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected


![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)

![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)



In [ ]:
# | export


def _update_path_in_markdown(root_path: str, doc_path: Path):
    """Update guide images relative path in the markdown files

    Args:
        root_path: Project's root directory
        doc_path: Path to the mkdocs/docs directory
    """
    files = _get_files_to_convert_to_markdown(root_path)

    for file in files:
        dir_prefix = str(file.parent)
        md = doc_path / f"{dir_prefix}" / f"{file.stem}.md"

        with open(Path(md), "r") as f:
            _new_text = f.read()
            _new_text = _replace_all(_new_text, dir_prefix)
        with open(Path(md), "w") as f:
            f.write(_new_text)


def _copy_images_to_docs_dir(root_path: str):
    """Copy guide images to the docs directory

    Args:
        root_path: Project's root directory
    """
    # Reference: https://github.com/quarto-dev/quarto-cli/blob/main/src/core/image.ts#L38
    image_extensions = [
        ".apng",
        ".avif",
        ".gif",
        ".jpg",
        ".jpeg",
        ".jfif",
        ".pjpeg",
        ".pjp",
        ".png",
        ".svg",
        ".webp",
    ]

    cache = proc_nbs()
    nbs_images_path = [
        p for p in Path(cache).glob(r"**/*") if p.suffix in image_extensions
    ]

    if len(nbs_images_path) > 0:
        doc_path = Path(root_path) / "mkdocs" / "docs"
        img_path = Path(doc_path) / "images" / "nbs"
        for src_path in nbs_images_path:
            dir_prefix = str(src_path.parent)[len(str(cache)) + 1 :]
            dst_path = Path(img_path) / f"{dir_prefix}"
            dst_path.mkdir(exist_ok=True, parents=True)
            shutil.copy(src_path, dst_path)

        _update_path_in_markdown(root_path, doc_path)


In [ ]:
def copy_guides(src, dst):

    src = Path(src)
    dst = Path(dst)
    assert src.exists()
    src_guides = src / "nbs" / "guides"
    src_guides_len = len(src_guides.parts)

    dst_guides = dst / "nbs" / "guides"

    for ext in [".ipynb", ".png", ".jpeg", ".jpg"]:
        for src_f in src_guides.glob(f"**/*{ext}"):
            dst_parts = dst_guides.parts + src_f.parts[src_guides_len:]
            dst_f = Path(*dst_parts)
            dst_f.parent.mkdir(exist_ok=True, parents=True)

            print(f"{src_f}, {dst_f}")
            shutil.copyfile(src_f, dst_f)


In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)

    new(d)

    _generate_markdown_from_files(root_path=d)

    _copy_images_to_docs_dir(d)

    # check image files
    print("Checks for images:")
    imgs = list((Path(d) / "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)

    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (
        Path(d) / "mkdocs" / "docs" / "guides" / "Guide_01_End_To_End_Walkthrough.md"
    ).exists()

    with open(
        (Path(d) / "mkdocs" / "docs" / "guides" / "Guide_01_End_To_End_Walkthrough.md"),
        "r",
    ) as f:
        contents = f.read()
        assert (
            "![Empty Git Repo](../images/nbs/guides/images/empty_git_repo.png)"
            in contents
        )
        assert (
            "![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)"
            in contents
        )
        assert "![](../images/nbs/guides/images/jupyter_home.png)" in contents

    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "guides"
        / "images"
        / "jupyter_home.png"
    ).exists()
    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "guides"
        / "images"
        / "empty_git_repo.png"
    ).exists()

    print("OK.")


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md



Checks for images:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/say_hello.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/git_repo_clone_page.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/CLI_command.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/foo_doc_string.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/guide_notebook.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/jupyter_home.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/hello_class.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq/mkdocs/docs/images/nbs/guides/images/guide_3.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpk02r6ydq

In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    _generate_markdown_from_files(root_path=d)
    _copy_images_to_docs_dir(d)

    # check image files
    print("Checks for images:")
    imgs = list((Path(d) / "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)

    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (
        Path(d) / "mkdocs" / "docs" / "guides" / "Guide_01_End_To_End_Walkthrough.md"
    ).exists()
    assert (
        Path(d) / "mkdocs" / "docs" / "blogs" / "Guide_01_End_To_End_Walkthrough.md"
    ).exists()

    with open(
        (Path(d) / "mkdocs" / "docs" / "blogs" / "Guide_01_End_To_End_Walkthrough.md"),
        "r",
    ) as f:
        contents = f.read()
        assert (
            "![Empty Git Repo](../images/nbs/blogs/images/empty_git_repo.png)"
            in contents
        )
        assert (
            "![Git Repo_Clone_Page](../images/nbs/blogs/images/git_repo_clone_page.png)"
            in contents
        )
        assert "![](../images/nbs/blogs/images/jupyter_home.png)" in contents

    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "blogs"
        / "images"
        / "jupyter_home.png"
    ).exists()
    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "guides"
        / "images"
        / "empty_git_repo.png"
    ).exists()

    print("OK.")


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metad

Checks for images:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/say_hello.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/git_repo_clone_page.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/CLI_command.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/foo_doc_string.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/guide_notebook.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/jupyter_home.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/hello_class.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/docs/images/nbs/blogs/images/guide_3.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyvaldjcg/mkdocs/

### Build summary for guides

In [ ]:
# | export


def _get_title_from_notebook(file_path: Path) -> str:
    cache = proc_nbs()
    _file_path = Path(cache) / file_path

    if not _file_path.exists():
        typer.secho(
            f"Unexpected error: path {_file_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    if _file_path.suffix == ".ipynb":
        nbp = NBProcessor(_file_path, procs=FrontmatterProc)
        nbp.process()
        title = nbp.nb.frontmatter_["title"]

    else:
        with open(_file_path) as f:
            contents = f.read()
        metadata = _fm2dict(contents, nb=False)
        metadata = {k.lower(): v for k, v in metadata.items()}
        title = metadata["title"]

    return title


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    
    create_sample_qmd_file(d)

    new(d)

    _generate_markdown_from_files(d)
    
    !ls -la {d}/nbs

    nb_paths = [
        Path("index.ipynb"),
        Path("sample.qmd"),
        Path("guides/Guide_01_End_To_End_Walkthrough.ipynb"),
    ]

    expected = ["Getting Started", "Sample", "End-To-End Walkthrough"]
    actual = []
    for nb_path in nb_paths:
        actual.append(_get_title_from_notebook(nb_path))

    print(actual)

    assert actual == expected


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpzqtxgygz/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o sample.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
metadata
  title: Sample
  author: Sample
  date: last-modified
  
Output created: _docs/sample.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  


total 48
drwxr-xr-x   7 harishm  staff    224 Dec 20 10:14 .
drwx------  11 harishm  staff    352 Dec 20 10:14 ..
-rw-r--r--   1 harishm  staff    290 Dec 20 10:14 _quarto.yml
drwxr-xr-x   5 harishm  staff    160 Dec 20 10:14 guides
-rw-r--r--   1 harishm  staff  12157 Dec 20 10:14 index.ipynb
-rw-r--r--   1 harishm  staff    191 Dec 20 10:14 nbdev.yml
-rw-r--r--   1 harishm  staff     77 Dec 20 10:14 sample.qmd


Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md



['Getting Started', 'Sample', 'End-To-End Walkthrough']


In [ ]:
# | export

def _get_sidebar_from_config(file_path: Path) -> List[Union[str, Any]]:
    
    if not file_path.exists():
        typer.secho(
            f"Path '{file_path.resolve()}' does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:
        with open(file_path) as f:
            config = yaml.safe_load(f)
        sidebar = config["website"]["sidebar"]["contents"]
    except KeyError as e:
        typer.secho(
            f"Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)
        
    return sidebar
    

def _read_sidebar_from_yml(root_path: str) -> List[Union[str, Any]]:
    _proc_dir = Path(root_path) / "_proc"
    sidebar_yml_path = _proc_dir / "sidebar.yml"
    _quarto_yml_path = _proc_dir / "_quarto.yml"

    custom_sidebar = get_value_from_config(root_path, "custom_sidebar")
    if custom_sidebar == "False":
        cmd = f'cd "{root_path}" && nbdev_docs'
        _sprun(cmd)
        
    return (
        _get_sidebar_from_config(sidebar_yml_path) 
        if sidebar_yml_path.exists()
        else _get_sidebar_from_config(_quarto_yml_path) 
    )

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    create_sample_qmd_file(d)
    new(d)

    with set_cwd(d):
        sidebar = _read_sidebar_from_yml(d)
    print(sidebar)
    assert sidebar == ["index.ipynb", "sample.qmd"]


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpgf4038v9/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpgf4038v9/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpgf4038v9/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpgf4038v9/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

[1/2] sample.qmd
[2/2] index.ipynb


['index.ipynb', 'sample.qmd']



Output created: _docs/index.html



In [ ]:
with pytest.raises(typer.Exit) as e:
    with TemporaryDirectory() as d:
        run_nbdev_new(d)

        new(d)

        updater = ConfigUpdater()
        updater.read(Path(d) / "settings.ini")
        updater["DEFAULT"]["custom_sidebar"] = True
        updater.update_file()
        with set_cwd(d):
            sidebar = _read_sidebar_from_yml(d)
        print(sidebar)
        assert sidebar == ["index.ipynb"]

print("OK")


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp53hrpx_8/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp53hrpx_8/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp53hrpx_8/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp53hrpx_8/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml.


OK


In [ ]:
# | export

def _flattern_sidebar_items(items: List[Union[str, Any]]) -> List[Union[str, Any]]:
    return (
        [ i for item in items if isinstance(item,list) for i in item] 
        + 
        [item for item in items if not isinstance(item,list)]
    )

def _expand_sidebar_if_needed(
    root_path: str, 
    sidebar: List[Union[str, Any]]
) -> List[Union[str, Any]]:
    """
    """
    _proc_dir = Path(root_path) / "_proc"
    exts = [".ipynb", ".qmd"]
    
    for index, item in enumerate(sidebar):
        if "auto" in item:
            files = list(_proc_dir.glob("".join(item["auto"].split('/')[1:]))) # type: ignore
            files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts]) # type: ignore
            sidebar[index] = files 
            
        if isinstance(item, dict) and "contents" in item:
            _contents = item["contents"]
            if isinstance(_contents, str) and bool(re.search(r'[*?\[\]]', _contents)):
                files = list(_proc_dir.glob(item["contents"]))
                files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts ]) # type: ignore
                item["contents"] = files 
    
    flat_sidebar = _flattern_sidebar_items(sidebar)
    return flat_sidebar

In [ ]:
_sidebar = """- auto: \"/*.ipynb\"
- section: Blogs
  contents: blogs/*
- section: Guides
  contents: guides/*
- section: Explanations
  contents:
  - explanations/explanation_1.ipynb
  - explanations/explanation_2.ipynb
"""

expected = [
    'getting_started.ipynb', 
    'index.ipynb',
    {'section': 'Blogs', 'contents': ['blogs/blog_1.ipynb', 'blogs/blog_2.ipynb']},
    {'section': 'Guides', 'contents': ['guides/Guide_01_End_To_End_Walkthrough.ipynb']},
    {'section': 'Explanations', 'contents': ['explanations/explanation_1.ipynb', 'explanations/explanation_2.ipynb']}
]


with TemporaryDirectory() as d:
    run_nbdev_new(d)
    _root_path = Path(".") if Path("settings.ini").exists()  else Path("..")
    copy_guides(_root_path, d)
    
    shutil.copyfile(
         Path(d) / "nbs" / "index.ipynb", Path(d) / "nbs" / "getting_started.ipynb"
    )
    
    (Path(d) / "nbs" / "blogs").mkdir(parents=True)
    for i in ["blog_1", "blog_2"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb", (Path(d) / "nbs" / "blogs" / f"{i}.ipynb")
        )
        
    (Path(d) / "nbs" / "explanations").mkdir(parents=True)
    for i in ["explanation_1", "explanation_2"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb", (Path(d) / "nbs" / "explanations" / f"{i}.ipynb")
        )
    
    
    new(d)

    #update _quarto.yml file
    _quarto_yml_path = Path(d) / "nbs" / "_quarto.yml"
    _yaml = YAML()
    config = _yaml.load(_quarto_yml_path)
    config["website"]["sidebar"]["contents"] = _yaml.load(_sidebar)
    _yaml.dump(config, _quarto_yml_path)
    
    #Set the custom_sidebar flag to True
    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["custom_sidebar"] = True
    updater.update_file()
    
    _generate_markdown_from_files(d)
    _copy_images_to_docs_dir(d)
    
    sidebar = _read_sidebar_from_yml(d)

    actual = _expand_sidebar_if_needed(d, sidebar)
    
    print(actual)
    assert actual == expected, actual

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyki3vyz0/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o getting_started.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: getting_started.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/getting_started.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../blog_1.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: blog_1.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: ../_docs/blog_1.md

pandoc -o ../blog_2.md
  to: >-
    c

['getting_started.ipynb', 'index.ipynb', {'section': 'Blogs', 'contents': ['blogs/blog_1.ipynb', 'blogs/blog_2.ipynb']}, {'section': 'Guides', 'contents': ['guides/Guide_01_End_To_End_Walkthrough.ipynb']}, {'section': 'Explanations', 'contents': ['explanations/explanation_1.ipynb', 'explanations/explanation_2.ipynb']}]


In [ ]:
# | export


def _filter_sidebar(
    root_path: str,
    sidebar: List[Union[str, Any]], 
    nbs_to_include: List[Path],
) -> List[Union[str, Any]]:
    nbs_to_include_set = set(map(str, nbs_to_include))
    _sidebar = _expand_sidebar_if_needed(root_path, sidebar)

    def should_include_item(item):
        if isinstance(item, str):
            return item in nbs_to_include_set
        elif isinstance(item, dict):
            return any(map(should_include_item, item["contents"]))

    return [item for item in _sidebar if should_include_item(item)]


In [ ]:
items_to_include = [
    Path("index.ipynb"),
    Path("blogs/Guide_01_End_To_End_Walkthrough.ipynb"),
    Path("guides/Guide_01_End_To_End_Walkthrough.ipynb"),
]
sidebar = [
    "index.ipynb",
    "Mkdocs.ipynb",
    "Social_Image_Generator.ipynb",
    {"section": "api", "contents": ["api/Mkdocs.ipynb"]},
    {"section": "blogs", "contents": ["blogs/Guide_01_End_To_End_Walkthrough.ipynb"]},
    {"section": "guides", "contents": ["guides/Guide_01_End_To_End_Walkthrough.ipynb"]},
]

expected = [
    "index.ipynb",
    {"section": "blogs", "contents": ["blogs/Guide_01_End_To_End_Walkthrough.ipynb"]},
    {"section": "guides", "contents": ["guides/Guide_01_End_To_End_Walkthrough.ipynb"]},
]
actual = _filter_sidebar(d, sidebar, items_to_include)
print(actual)

assert actual == expected


['index.ipynb', {'section': 'blogs', 'contents': ['blogs/Guide_01_End_To_End_Walkthrough.ipynb']}, {'section': 'guides', 'contents': ['guides/Guide_01_End_To_End_Walkthrough.ipynb']}]


In [ ]:
# | export


def _generate_nav_from_sidebar(sidebar_items, level=0):
    output = ""
    links = [
        "{}- [{}]({}.md)\n".format(
            "    " * level,
            _get_title_from_notebook(Path(item)),
            Path(item).with_suffix(""),
        )
        if isinstance(item, str)
        else "{}- {}\n".format("    " * level, item["section"])
        + _generate_nav_from_sidebar(item["contents"], level + 1)
        for item in sidebar_items
    ]
    output += "".join(links)
    return output


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    create_sample_qmd_file(d)

    new(d)
    with set_cwd(d):
        sidebar = _read_sidebar_from_yml(d)
        nbs_to_include = _get_files_to_convert_to_markdown(d)

        filtered_sidebar = _filter_sidebar(d, sidebar, nbs_to_include)
        print(filtered_sidebar)
        assert filtered_sidebar == [
            "index.ipynb",
            "sample.qmd",
            {
                "section": "blogs",
                "contents": ["blogs/Guide_01_End_To_End_Walkthrough.ipynb"],
            },
            {
                "section": "guides",
                "contents": ["guides/Guide_01_End_To_End_Walkthrough.ipynb"],
            },
        ]

        actual = _generate_nav_from_sidebar(filtered_sidebar)
        print(actual)

        expected = """- [Getting Started](index.md)
- [Sample](sample.md)
- blogs
    - [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)
- guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)
"""

        assert actual == expected, actual


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiaog66ac/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

[1/7] blogs/Guide_01_End_To_End_Walkthrough.ipynb
[2/7] Mkdocs.ipynb
[3/7] guides/Guide_01_End_To_End_Walkthrough.ipynb
[4/7] api/Mkdocs.ipynb
[5/7] sample.qmd
[6/7] index.ipynb
[7/7] Social_Image_Generator.ipynb



['index.ipynb', 'sample.qmd', {'section': 'blogs', 'contents': ['blogs/Guide_01_End_To_End_Walkthrough.ipynb']}, {'section': 'guides', 'contents': ['guides/Guide_01_End_To_End_Walkthrough.ipynb']}]


Output created: _docs/index.html



- [Getting Started](index.md)
- [Sample](sample.md)
- blogs
    - [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)
- guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)



In [ ]:
# | export


def _generate_summary_for_sidebar(
    root_path: str,
) -> str:
    with set_cwd(root_path):
        sidebar = _read_sidebar_from_yml(root_path)
        nbs_to_include = _get_files_to_convert_to_markdown(root_path)

        filtered_sidebar = _filter_sidebar(root_path, sidebar, nbs_to_include)
        sidebar_nav = _generate_nav_from_sidebar(filtered_sidebar)

        return sidebar_nav


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    # copy nbs folder
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    copy_guides(_root_path, d)
    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    cmd = f'cd "{d}" && nbdev_sidebar'
    print(f"executing the command: {cmd}")
    _sprun(cmd)

    new(d)

    actual = _generate_summary_for_sidebar(d)

    print(actual)

    expected = """- [Getting Started](index.md)
- blogs
    - [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)
- guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)
"""

    assert actual == expected, actual


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/README.md



../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwq8ixkq0/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

[1/6] blogs/Guide_01_End_To_End_Walkthrough.ipynb
[2/6] Mkdocs.ipynb
[3/6] guides/Guide_01_End_To_End_Walkthrough.ipynb
[4/6] api/Mkdocs.ipynb
[5/6] index.ipynb
[6/6] Social_Image_Generator.ipynb

Output created: _docs/index.html



- [Getting Started](index.md)
- blogs
    - [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)
- guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)



### Build API

In [ ]:
# | export


def get_submodules(package_name: str) -> List[str]:
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(package_name)
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules


In [ ]:
submodules = get_submodules("mkdocs")
submodules


['mkdocs.commands',
 'mkdocs.commands.babel',
 'mkdocs.commands.build',
 'mkdocs.commands.gh_deploy',
 'mkdocs.commands.new',
 'mkdocs.commands.serve',
 'mkdocs.commands.setup',
 'mkdocs.config',
 'mkdocs.config.base',
 'mkdocs.config.config_options',
 'mkdocs.config.defaults',
 'mkdocs.contrib',
 'mkdocs.contrib.search',
 'mkdocs.contrib.search.search_index',
 'mkdocs.exceptions',
 'mkdocs.livereload',
 'mkdocs.localization',
 'mkdocs.plugins',
 'mkdocs.structure',
 'mkdocs.structure.files',
 'mkdocs.structure.nav',
 'mkdocs.structure.pages',
 'mkdocs.structure.toc',
 'mkdocs.tests',
 'mkdocs.tests.base',
 'mkdocs.tests.build_tests',
 'mkdocs.tests.cli_tests',
 'mkdocs.tests.config',
 'mkdocs.tests.config.base_tests',
 'mkdocs.tests.config.config_options_legacy_tests',
 'mkdocs.tests.config.config_options_tests',
 'mkdocs.tests.config.config_tests',
 'mkdocs.tests.gh_deploy_tests',
 'mkdocs.tests.integration',
 'mkdocs.tests.livereload_tests',
 'mkdocs.tests.localization_tests',
 'mkd

In [ ]:
# | export


def generate_api_doc_for_submodule(root_path: str, submodule: str) -> str:
    subpath = "API/" + submodule.replace(".", "/") + ".md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(f"::: {submodule}")
    subnames = submodule.split(".")
    if len(subnames) > 2:
        return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
    else:
        return f"- [{submodule}]({subpath})"


def generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    submodules = get_submodules(module_name)
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "API", ignore_errors=True)

    if not len(submodules):
        return ""

    submodule_summary = "\n".join(
        [
            generate_api_doc_for_submodule(root_path=root_path, submodule=x)
            for x in submodules
        ]
    )

    return "- API\n" + textwrap.indent(submodule_summary, prefix=" " * 4)


In [ ]:
with TemporaryDirectory() as d:
    api_summary = generate_api_docs_for_module(d, "nbdev")
    print(api_summary)

    # make sure all paths exist
    paths = re.findall("\(.*?\)", api_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path


- API
    - [nbdev.clean](API/nbdev/clean.md)
    - [nbdev.cli](API/nbdev/cli.md)
    - [nbdev.config](API/nbdev/config.md)
    - [nbdev.doclinks](API/nbdev/doclinks.md)
    - [nbdev.export](API/nbdev/export.md)
    - [nbdev.extract_attachments](API/nbdev/extract_attachments.md)
    - [nbdev.frontmatter](API/nbdev/frontmatter.md)
    - [nbdev.imports](API/nbdev/imports.md)
    - [nbdev.maker](API/nbdev/maker.md)
    - [nbdev.merge](API/nbdev/merge.md)
    - [nbdev.migrate](API/nbdev/migrate.md)
    - [nbdev.process](API/nbdev/process.md)
    - [nbdev.processors](API/nbdev/processors.md)
    - [nbdev.qmd](API/nbdev/qmd.md)
    - [nbdev.quarto](API/nbdev/quarto.md)
    - [nbdev.release](API/nbdev/release.md)
    - [nbdev.serve](API/nbdev/serve.md)
    - [nbdev.serve_drv](API/nbdev/serve_drv.md)
    - [nbdev.showdoc](API/nbdev/showdoc.md)
    - [nbdev.sync](API/nbdev/sync.md)
    - [nbdev.test](API/nbdev/test.md)


In [ ]:
# | export


def _restrict_line_length(s: str, width: int = 80) -> str:
    """Restrict the line length of the given string.

    Args:
        s: Docstring to fix the width
        width: The maximum allowed line length

    Returns:
        A new string in which each line is less than the specified width.
    """
    _s = ""

    for blocks in s.split("\n\n"):
        sub_block = blocks.split("\n  ")
        for line in sub_block:
            line = line.replace("\n", " ")
            line = "\n".join(textwrap.wrap(line, width=width, replace_whitespace=False))
            if len(sub_block) == 1:
                _s += line + "\n\n"
            else:
                _s += "\n" + line + "\n" if line.endswith(":") else " " + line + "\n"
    return _s


In [ ]:
s = """usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory in the **root_path** directory and populates it with
initial values. You should edit mkdocs.yml file to customize it if needed.

positional arguments:
  root_path

optional arguments:
  -h, --help  show this help message and exit show this help message and exit show this help message and exit
  -h, --help  show this help message and exit
  --port PORT
  --domain DOMAIN
"""

width = 60
doc = _restrict_line_length(s, width)

print(doc)
assert all([len(line) <= width for line in doc.splitlines() if line.strip() != ""])


usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory
in the **root_path** directory and populates it with initial
values. You should edit mkdocs.yml file to customize it if
needed.


positional arguments:
 root_path

optional arguments:
 -h, --help  show this help message and exit show this help
message and exit show this help message and exit
 -h, --help  show this help message and exit
 --port PORT
 --domain DOMAIN



In [ ]:
# | export


def generate_cli_doc_for_submodule(root_path: str, cmd: str) -> str:

    cli_app_name = cmd.split("=")[0]
    module_name = cmd.split("=")[1].split(":")[0]
    method_name = cmd.split("=")[1].split(":")[1]

    subpath = f"CLI/{cli_app_name}.md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)

    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(module_name)
    if isinstance(getattr(m, method_name), typer.Typer):
        app = typer.Typer()
        app.command()(generate_cli_doc)
        runner = CliRunner()
        result = runner.invoke(app, [module_name, cli_app_name])
        cli_doc = str(result.stdout)
    else:
        cmd = f"{cli_app_name} --help"

        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        cli_doc = subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
            cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        ).stdout.decode("utf-8")

        cli_doc = _restrict_line_length(cli_doc)
        cli_doc = "\n```\n" + cli_doc + "\n```\n"

    with open(path, "w") as f:
        f.write(cli_doc)

    return f"- [{cli_app_name}]({subpath})"


def generate_cli_docs_for_module(root_path: str, module_name: str) -> str:
    shutil.rmtree(Path(root_path) / "mkdocs" / "docs" / "CLI", ignore_errors=True)
    console_scripts = get_value_from_config(root_path, "console_scripts")

    if not console_scripts:
        return ""

    submodule_summary = "\n".join(
        [
            generate_cli_doc_for_submodule(root_path=root_path, cmd=cmd)
            for cmd in console_scripts.split("\n")
        ]
    )

    return "- CLI\n" + textwrap.indent(submodule_summary, prefix=" " * 4)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    cli_summary = generate_cli_docs_for_module(d, "nbdev_mkdocs")
    print(cli_summary)

    # make sure all paths and content exist
    paths = re.findall("\(.*?\)", cli_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6f0u_8nx/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6f0u_8nx/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6f0u_8nx/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6f0u_8nx/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md



- CLI
    - [nbdev_mkdocs](CLI/nbdev_mkdocs.md)
    - [nbdev_mkdocs_new](CLI/nbdev_mkdocs_new.md)
    - [nbdev_mkdocs_prepare](CLI/nbdev_mkdocs_prepare.md)
    - [nbdev_mkdocs_preview](CLI/nbdev_mkdocs_preview.md)
    - [nbdev_mkdocs_docs](CLI/nbdev_mkdocs_docs.md)


In [ ]:
# | export


def _copy_change_log_if_exists(
    root_path: Union[Path, str], docs_path: Union[Path, str]
) -> str:
    changelog = ""
    source_change_log_path = Path(root_path) / "CHANGELOG.md"
    dst_change_log_path = Path(docs_path) / "CHANGELOG.md"
    if source_change_log_path.exists():
        shutil.copy(source_change_log_path, dst_change_log_path)
        changelog = "- [Releases](CHANGELOG.md)"
    return changelog


In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == ""

    change_log_path = Path(d) / "CHANGELOG.md"
    with open(change_log_path, "w") as f:
        f.write("CHANGELOG")

    (Path(d) / "mkdocs" / "docs").mkdir(exist_ok=True, parents=True)
    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "- [Releases](CHANGELOG.md)"


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements already added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpu6q4m23i/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpu6q4m23i/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpu6q4m23i/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpu6q4m23i/mkdocs/summary_template.txt' generated.
change_log=
change_log=- [Releases](CHANGELOG.md)


Output created: _docs/README.md



### Brining it all together

In [ ]:
# | export


def build_summary(
    root_path: str,
    module: str,
):
    # create docs_path if needed
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    # generate markdown files
    _generate_markdown_from_files(root_path)

    # copy images to docs dir and update path in generated markdown files
    _copy_images_to_docs_dir(root_path)

    # generates sidebar navigation
    sidebar = _generate_summary_for_sidebar(root_path)

    # generate API
    api = generate_api_docs_for_module(root_path, module)

    # generate CLI
    cli = generate_cli_docs_for_module(root_path, module)

    # copy CHANGELOG.md as CHANGELOG.md is exists
    changelog = _copy_change_log_if_exists(root_path, docs_path)

    # read summary template from file
    with open(Path(root_path) / "mkdocs" / "summary_template.txt") as f:
        summary_template = f.read()

    summary = summary_template.format(
        sidebar=sidebar, api=api, cli=cli, changelog=changelog
    )
    summary = "\n".join(
        [l for l in [l.rstrip() for l in summary.split("\n")] if l != ""]
    )

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    copy_guides(_root_path, d)
    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(
            _root_path / "nbs" / f,
            Path(d) / "nbs" / f
        )
        
    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
            Path(d) / "nbs" / "Mkdocs.ipynb",
            (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
        )
    
    shutil.copytree(
        (Path(d) / "nbs" / "guides"),
        (Path(d) / "nbs" / "blogs")
    )
    
    
    new(d)
    
    !cat {d}/nbs/sidebar.yml
    
    # todo: handle if the sidebar.yml dosen't exists
    build_summary(d, "nbdev_mkdocs")

    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)
    assert "- [Getting Started](index.md)" in summary
    assert "- [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)" in summary
    assert "- [Releases](CHANGELOG.md)" in summary


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpkcpkm47n/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metad

- [Getting Started](index.md)
- blogs
    - [End-To-End Walkthrough](blogs/Guide_01_End_To_End_Walkthrough.md)
- guides
    - [End-To-End Walkthrough](guides/Guide_01_End_To_End_Walkthrough.md)
- API
    - [nbdev_mkdocs.docstring](API/nbdev_mkdocs/docstring.md)
    - [nbdev_mkdocs.mkdocs](API/nbdev_mkdocs/mkdocs.md)
    - [nbdev_mkdocs.social_image_generator](API/nbdev_mkdocs/social_image_generator.md)
- CLI
    - [nbdev_mkdocs](CLI/nbdev_mkdocs.md)
    - [nbdev_mkdocs_new](CLI/nbdev_mkdocs_new.md)
    - [nbdev_mkdocs_prepare](CLI/nbdev_mkdocs_prepare.md)
    - [nbdev_mkdocs_preview](CLI/nbdev_mkdocs_preview.md)
    - [nbdev_mkdocs_docs](CLI/nbdev_mkdocs_docs.md)
- [Releases](CHANGELOG.md)


### Copy CNAME if needed

In [ ]:
# | export


def copy_cname_if_needed(root_path: str):
    cname_path = Path(root_path) / "CNAME"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "CNAME"
    if cname_path.exists():
        dst_path.parent.mkdir(exist_ok=True, parents=True)
        shutil.copyfile(cname_path, dst_path)
        typer.secho(
            f"File '{cname_path.resolve()}' copied to '{dst_path.resolve()}'.",
        )
    else:
        typer.secho(
            f"File '{cname_path.resolve()}' not found, skipping copying..",
        )


In [ ]:
for has_cname in [True, False]:
    with TemporaryDirectory() as d:
        settings_path = Path(d) / "settings.ini"
        for fname in ["settings.ini", "README.md"] + ["CNAME"] if has_cname else []:
            shutil.copyfile(Path("..") / fname, Path(d) / fname)

        copy_cname_if_needed(d)
        if has_cname:
            assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
        else:
            assert not (Path(d) / "mkdocs" / "docs" / "CNAME").exists()


File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6r6c_qnu/CNAME' copied to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6r6c_qnu/mkdocs/docs/CNAME'.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpo3k1noza/CNAME' not found, skipping copying..


In [ ]:
# | export


def _copy_docs_overrides(root_path: str):
    """Copy lib assets inside mkodcs/docs directory

    Args:
        root_path: Project's root path.
    """
    src_path = Path(root_path) / "mkdocs" / "docs_overrides"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "overrides"

    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    shutil.rmtree(dst_path, ignore_errors=True)
    shutil.copytree(src_path, dst_path)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    assert (Path(d) / "mkdocs" / "docs_overrides").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js" / "extra.js").exists()

    _copy_docs_overrides(d)

    assert (Path(d) / "mkdocs" / "docs" / "overrides").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js" / "extra.js").exists()


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


Requirements added to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfn67gzla/settings.ini'.
Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfn67gzla/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfn67gzla/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfn67gzla/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md



In [ ]:
# | export


def nbdev_mkdocs_docs(root_path: str, refresh_quarto_settings: bool = False):
    """Prepares mkdocs documentation

    Args:
        root_path: Project's root path.
        refresh_quarto_settings: Flag to refresh quarto yml file. This flag should be set to `True`
            if this function is called directly without calling prepare.
    """

    with set_cwd(root_path):

        if refresh_quarto_settings:
            refresh_quarto_yml()

        copy_cname_if_needed(root_path)

        _copy_docs_overrides(root_path)

        lib_name = get_value_from_config(root_path, "lib_name")
        lib_path = get_value_from_config(root_path, "lib_path")

        build_summary(root_path, lib_path)
        #         _generate_default_social_image_link(root_path)

        cmd = f"mkdocs build -f \"{(Path(root_path) / 'mkdocs' / 'mkdocs.yml').resolve()}\""
        _sprun(cmd)


@call_parse
def nbdev_mkdocs_docs_cli(root_path: str = "."):
    """Prepares mkdocs documentation"""
    nbdev_mkdocs_docs(root_path, refresh_quarto_settings=True)


def prepare(root_path: str, no_test: bool = False):
    """Prepares mkdocs for serving

    Args:
        root_path: path under which mkdocs directory will be created
    """
    with set_cwd(root_path):

        if no_test:
            nbdev_export.__wrapped__()
            refresh_quarto_yml()
            nbdev_readme.__wrapped__(chk_time=True)
        else:
            cmd = "nbdev_prepare"
            _sprun(cmd)

    nbdev_mkdocs_docs(root_path)


@call_parse
def prepare_cli(root_path: str = "."):
    """Prepares mkdocs for serving"""
    prepare(root_path)


In [ ]:
# | notest

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)

    shutil.copyfile(
        Path(".") / "CNAME" if Path("settings.ini").exists() else Path("..") / "CNAME",
        Path(d) / "CNAME",
    )

    new(d)

    prepare(d)

    assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "guides").exists()
#     !ls {d}/mkdocs/docs


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md



File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/CNAME' copied to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6xa0us3w/mkdocs/docs/CNAME'.


pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

[1/2] guides/Guide_01_End_To_End_Walkthrough.ipynb
[2/2] index.ipynb

Output created: _docs/index.html

INFO     -  UserWarning: The 'python-legacy' extra of mkdocstrings will become mandatory in the next release. We have no way to detect if you already specify it, so if you do, please ignore this warning. You can gl

## Preview

In [ ]:
# | export


def preview(root_path: str, port: Optional[int] = None):
    """Previes mkdocs documentation

    Args:
        root_path: path under which mkdocs directory will be created
        port: port to use
    """
    with set_cwd(root_path):
        prepare(root_path=root_path, no_test=True)

        cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
        if port:
            cmd = cmd + f":{port}"

        with subprocess.Popen(  # nosec B603:subprocess_without_shell_equals_true
            shlex.split(cmd),
            stdout=subprocess.PIPE,
            bufsize=1,
            text=True,
            universal_newlines=True,
        ) as p:
            for line in p.stdout:  # type: ignore
                print(line, end="")

        if p.returncode != 0:
            typer.secho(
                f"Command cmd='{cmd}' failed!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(6)


@call_parse
def preview_cli(root_path: str = ".", port: Optional[int] = None):
    """Previes mkdocs documentation"""
    preview(root_path, port)


In [ ]:
# | notest

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    preview(d, port=4000)


settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  


../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/.ipynb_checkpoints/Guide_01_End_To_End_Walkthrough-checkpoint.ipynb
../nbs/guides/images/say_hello.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/images/say_hello.png
../nbs/guides/images/git_repo_clone_page.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/images/git_repo_clone_page.png
../nbs/guides/images/CLI_command.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/images/CLI_command.png
../nbs/guides/images/foo_doc_string.png, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_mkzhdiy/nbs/guides/images/foo_doc_string.png
../nbs/guides/images/guide_notebook.png, /var/folders/6n/3rjds7v

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Getting Started
  
Output created: _docs/index.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metadata
  title: End-To-End Walkthrough
  
Output created: ../_docs/Guide_01_End_To_End_Walkthrough.md

pandoc -o ../Guide_01_End_To_End_Walkthrough.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: guide_end_to_end_walkthrough.html
  standalone: true
  default-image-extension: png
  
metad

KeyboardInterrupt: 